In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt
import cupy as cp

In [ ]:
# Common scene parameters
nx, ny = 300, 150
aspect_ratio = nx / ny
fov = np.pi / 3.0
lower_left = np.array([-2.0, -1.0, -1.0])
horizontal = np.array([4.0, 0.0, 0.0])
vertical = np.array([0.0, 2.0, 0.0])
origin = np.array([0.0, 0.0, 0.0])
sphere_center = np.array([0.0, 0.0, -1.0])
sphere_radius = 0.5

def ray_color(ro, rd):
    # ray-sphere intersection
    oc = ro - sphere_center
    a = np.dot(rd, rd)
    b = np.dot(oc, rd)
    c = np.dot(oc, oc) - sphere_radius**2
    disc = b*b - a*c
    if disc > 0:
        t = (-b - np.sqrt(disc)) / a
        if t > 0.001:
            hit = ro + t*rd
            normal = (hit - sphere_center) / np.linalg.norm(hit - sphere_center)
            base = np.array([1.0, 1.0, 0.0])
            intensity = 0.5 * (normal[1] + 1.0)
            return intensity * base
    # sky
    udir = rd / np.linalg.norm(rd)
    t = 0.5 * (udir[1] + 1.0)
    return (1-t)*np.array([1.0,1.0,1.0]) + t*np.array([0.5,0.7,1.0])



In [ ]:
def render_cupy_vectorized():
    # define in form of cupy arrays
    lower_left = cp.array([-2.0, -1.0, -1.0])
    horizontal = cp.array([4.0, 0.0, 0.0])
    vertical = cp.array([0.0, 2.0, 0.0])
    origin = cp.array([0.0, 0.0, 0.0])
    sphere_center = cp.array([0.0, 0.0, -1.0])

    i = cp.arange(nx).reshape(1, nx)
    j = cp.arange(ny).reshape(ny, 1)
    u = i / nx
    v = j / ny

    dirs = lower_left[None,None,:] + u[:,:,None]*horizontal + v[:,:,None]*vertical - origin
    norms = cp.linalg.norm(dirs, axis=2, keepdims=True)
    dirs = dirs / norms

    # call ray_color on arrays
    oc = origin - sphere_center
    b = cp.sum(dirs * oc[None,None,:], axis=2)
    c = cp.sum(oc*oc) - sphere_radius**2
    disc = b*b - c * cp.sum(dirs*dirs, axis=2)
    mask = disc > 0

    t = (-b - cp.sqrt(disc)) / cp.sum(dirs*dirs, axis=2)
    hit = origin + dirs * t[:,:,None]
    normal = (hit - sphere_center) / cp.linalg.norm(hit - sphere_center, axis=2, keepdims=True)
    base = cp.array([1.0,1.0,0.0])
    intensity = 0.5 * (normal[:,:,1] + 1.0)
    col_sphere = (intensity[:,:,None] * base[None,None,:])
    udir = dirs / norms
    t_sky = 0.5 * (udir[:,:,1] + 1.0)
    col_sky = (1-t_sky)[:,:,None]*cp.array([1.0,1.0,1.0])[None,None,:] + t_sky[:,:,None]*cp.array([0.5,0.7,1.0])[None,None,:]
    image = cp.where(mask[:,:,None], col_sphere, col_sky)
    
    return cp.asnumpy(image)

In [ ]:
times = {}
start = time.perf_counter()
img = render_cupy_vectorized()
exec_time = (time.perf_counter() - start) * 1000  # ms
print(exec_time, "ms")
plt.imshow(img)
plt.axis('off')
plt.show()